In [12]:
import cv2 as cv
from imutils import resize
import numpy as np

In [24]:
""" Algorithm
* find the pixel absolute diffrence using absdiff
* convert to gray for simplyifing the process of finding contour
* middle process
    ** blur the frame
    ** threshold the gray image to cancel the unnecessary elements
    ** dilate the image
* find and draw contour
"""

def detectMotion(frame1, frame2) -> np.array:
    diff = cv.absdiff(frame1, frame2)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (3, 3), 0)
    _, thres = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
    dilated = cv.dilate(thres, None, iterations=1)
    contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
#     cv.drawContours(frame1, contours, -1, (0, 255, 0), 3)
    for contour in contours:
        (x, y, w, h) = cv.boundingRect(contour)
        if cv.contourArea(contour) >= 1000:
            cv.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 3)
            cv.putText(frame1, "Status: {}".format("Movement"), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
    
    return frame1

## Tracking object using meanshift

In [51]:
cap = cv.VideoCapture("./Videos/People Walking Free Stock Footage, Royalty-Free No Copyright Content.mp4")
_, frame = cap.read()
x, y, w, h = 125, 235, 45, 45
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((1., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 2)

while cap.isOpened():
#     frame = detectMotion(frame1, frame2)
    _, frame = cap.read()
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    
    ret, track_window = cv.meanShift(dst, track_window, term_crit)
    x, y, w, h = track_window
    final_image = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
    cv.imshow("video", final_image)
    
    if cv.waitKey(20) == ord("q"):
        break
cv.destroyAllWindows()

## CAMShift

In [55]:
cap = cv.VideoCapture("./Videos/People Walking Free Stock Footage, Royalty-Free No Copyright Content.mp4")
_, frame = cap.read()
x, y, w, h = 125, 235, 45, 45
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((1., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 2)

while cap.isOpened():
#     frame = detectMotion(frame1, frame2)
    _, frame = cap.read()
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    ret, track_window = cv.CamShift(dst, track_window, term_crit)
    
    pts = cv.boxPoints(ret)
    pts = np.int0(pts)
    final_image = cv.polylines(frame, [pts], True, (0, 255, 0), 2)
#     x, y, w, h = track_window
#     final_image = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
    cv.imshow("video", final_image)
    
    if cv.waitKey(20) == ord("q"):
        break
cv.destroyAllWindows()